<a href="https://colab.research.google.com/github/Harini0324/ECOLOPPER/blob/main/EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv("/content/vegetation_dataset.csv")

features = df[["contour_area",  "bounding_box_width", "bounding_box_height", "mean_intensity", "standard_deviation", "contrast",  "energy"]]
target = df["height"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.35, random_state=42)

rf_model = RandomForestRegressor(n_estimators=120, max_depth=None, random_state=0)
rf_model.fit(X_train, y_train)

predicted_heights = rf_model.predict(X_test)

mse = mean_squared_error(y_test, predicted_heights)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicted_heights)
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")


Mean Squared Error (MSE): 0.047161135262486664
Root Mean Squared Error (RMSE): 0.21716614667688577
Mean Absolute Error (MAE): 0.1735591118240549


In [ ]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=128, max_depth=6, learning_rate=0.1)
xgb_model.fit(X_train, y_train)

predicted_heights = xgb_model.predict(X_test)

mse = mean_squared_error(y_test, predicted_heights)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicted_heights)
print(f"XGBoost - Mean Squared Error (MSE): {mse}")
print(f"XGBoost - Root Mean Squared Error (RMSE): {rmse}")
print(f"XGBoost - Mean Absolute Error (MAE): {mae}")


XGBoost - Mean Squared Error (MSE): 0.04969591610051103
XGBoost - Root Mean Squared Error (RMSE): 0.22292580851151136
XGBoost - Mean Absolute Error (MAE): 0.1788050644865506


In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(n_estimators=133, max_depth=5, num_leaves=7,learning_rate=0.1)
lgb_model.fit(X_train, y_train)

predicted_heights = lgb_model.predict(X_test)

mse = mean_squared_error(y_test, predicted_heights)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predicted_heights)
print(f"LightGBM - Mean Squared Error (MSE): {mse}")
print(f"LightGBM - Root Mean Squared Error (RMSE): {rmse}")
print(f"LightGBM - Mean Absolute Error (MAE): {mae}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000289 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 3250, number of used features: 7
[LightGBM] [Info] Start training from score 17.267268
LightGBM - Mean Squared Error (MSE): 0.04922006247356733
LightGBM - Root Mean Squared Error (RMSE): 0.22185594982683546
LightGBM - Mean Absolute Error (MAE): 0.1776817188346781


In [ ]:
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge

lgbm_model = lgb_model
ridge_model = rf_model

models = [('lgbm', lgbm_model), ('xgb', xgb_model), ('ridge', ridge_model)]

weights = [1.0, 0.8, 0.5]

voting_model = VotingRegressor(estimators=models, weights=weights)
voting_model.fit(features, target)

predictions = voting_model.predict(features)

from sklearn.metrics import mean_squared_error, r2_score

predictions = voting_model.predict(features)

mse = mean_squared_error(target, predictions)
print("Mean Squared Error (MSE):", mse)

r2 = r2_score(target, predictions)
print("R-squared:", r2)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000652 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 7
[LightGBM] [Info] Start training from score 17.214614
Mean Squared Error (MSE): 0.02209824134038749
R-squared: 0.9997613801353388


In [ ]:
an Squared Error (MSE): 0.02209824134038749
R-squared: 0.9997613801353388

In [ ]:
import cv2
import numpy as np
import pandas as pd


def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if len(image.shape) > 2 else image

    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No object detected in the image!")
        return None

    cnt = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    x, y, width, height = cv2.boundingRect(cnt)
    mean_intensity = cv2.mean(gray)[0]
    std = np.std(gray)
    texture_energy = np.var(gray)
    texture_homogeneity = np.std(texture_energy)
    contrast=np.std(mean_intensity)
    return area, perimeter, width, height, mean_intensity, std,contrast ,texture_energy, texture_homogeneity


image = cv2.imread("tree12.jpg")
contour_area,contour_perimeter, bounding_box_width, bounding_box_height,mean_intensity, standard_deviation, contrast,  energy,homogeneity= extract_features(image)

features_df = pd.DataFrame([[contour_area,bounding_box_width, bounding_box_height, mean_intensity, standard_deviation, contrast,  energy]],
                           columns=["contour_area",  "bounding_box_width", "bounding_box_height",
                                  "mean_intensity", "standard_deviation", "contrast",  "energy"])

predicted_height = voting_model.predict(features_df)[0]

print("Predicted vegetation height:", predicted_height)

Predicted vegetation height: 33.575575006533136
